<a href="https://colab.research.google.com/github/tetsufmbio/remolog/blob/main/notebooks/remolog_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Remolog: identifying remote homologs using structural alignment data

## Quick start

1. Press "Runtime" --> "Run all".
2. In the next cell (*Upload protein structure*), it will appear the bottom "Choose file". Click on it and choose one or more pdb file to be uploaded and analyzed by the pipeline*.
3. After the running, it will download a file named "final_result.tab". See its description in the end of this notebook.
4. Running time of this pipeline in Colab take ~10 min when using foldseek for screening**.

\* If you have only the amino acid sequence, you can predict its structure using the [AlphaFold Colab notebook](https://github.com/sokrypton/ColabFold)

\** If no remote homolog was found, try to increase the parameter "n".


In [ ]:
%cd /content
! mkdir input
%cd input

from google.colab import files

uploaded = files.upload()
#@markdown  By running this cell, a bottom "Choose files" may appear. Click on it and choose the pdb file to be analyzed (you may upload multiple files).

%cd /content


## Setting parameters

In [1]:
screening = "foldseek" #@param ["foldseek","tmalign", "fatcat"]
#@markdown  - software to be used to retrieve structurely similar proteins in SCOPe database.

n = 20 #@param {type:"integer"}
#@markdown  - analyze n most similar protein structures.

database = "scope40" #@param ["scope40"]
#@markdown  - protein structure database to be used.

jobName = "final_result" #@param {type: "string"}
#@markdown  - prefix for the final output name.


## Installing dependencies and downloading database

In [ ]:
# setup environment variables
import os
os.environ['FATCAT'] = '/content/programs/FATCAT-dist'
os.environ['PATH'] += ':/content/programs/FATCAT-dist/FATCATMain:/content/bin'
os.environ['HEADN'] = str(n)
os.environ['SCREEN'] = screening

In [ ]:
%cd /content/
! if [ ! -d bin ]; then mkdir bin; fi
! if [ ! -d programs ]; then mkdir programs; fi
! if [ ! -d view ]; then mkdir view; fi

In [ ]:
# download and install TMalign
%cd /content/bin
! if [ ! -e TMalign ]; then \
    wget "https://zhanggroup.org/TM-align/TMalign.cpp"; \
    g++ -static -O3 -ffast-math -lm -o TMalign TMalign.cpp; \
  fi


In [ ]:
# download and install FATCAT
%cd /content/programs
! if [ ! -d FATCAT-dist ]; then \
    git clone https://github.com/GodzikLab/FATCAT-dist.git; \
    cd FATCAT-dist/; ./Install; \
  fi


In [ ]:
# download and install lovoalign
%cd /content/bin
! if [ ! -e lovoalign ]; then \
    wget "https://github.com/m3g/lovoalign/archive/refs/tags/22.0.0.tar.gz"; \
    tar -xzf 22.0.0.tar.gz; cd lovoalign-22.0.0/src; make; cp ../bin/lovoalign /content/bin; \
  fi

In [ ]:
# download some scripts and model
%cd /content/programs
! if [ ! -d remolog ]; then \
    git clone https://github.com/tetsufmbio/remolog.git; \
  fi

In [ ]:
%cd /content/programs
! if [ ! -e /content/bin/foldseek ]; then \
    wget https://mmseqs.com/foldseek/foldseek-linux-sse41.tar.gz; tar xvzf foldseek-linux-sse41.tar.gz; \
    cp foldseek/bin/foldseek /content/bin; \
  fi

In [ ]:
# download and format scope40 database
%cd /content
! if [ ! -d database ]; then \
    mkdir database; cd database; \
    wget https://scop.berkeley.edu/downloads/pdbstyle/pdbstyle-sel-gs-bib-40-2.08.tgz; \
    tar -zxf pdbstyle-sel-gs-bib-40-2.08.tgz; mv pdbstyle-2.08/*/*.ent . ; rm -rf pdbstyle*; \
    ls *.ent > ../list_scope.tab; for i in *.ent; do mv $i $i.pdb; done; \
  fi

In [ ]:
# creating foldseek database
%cd /content
! if [ ! -d foldseek_data ]; then \
  mkdir foldseek_data; cd /content/foldseek_data; \
  foldseek createdb /content/database/ fs_data ; \
fi


## Running protein structure alignment

In [ ]:
%cd /content
! if [ -d result ]; then rm -rf result ; fi
! mkdir result;
! mkdir result/screening;

In [ ]:
# screening for similar proteins using foldseek
%cd /content/input

! if [ $SCREEN = "foldseek" ]; then \
    for f in *; do \
      foldseek easy-search $f /content/foldseek_data/fs_data /content/result/screening/tmp.tab.fmt /content/tmpFolder --max-seqs $HEADN -e inf; \
      cut -f 1,2 /content/result/screening/tmp.tab.fmt | uniq | perl -ne '@a = split(/\./, $_); print join(".", @a[0 .. $#a-2])."\n";' > /content/result/screening/$f.tab.fmt; \
      rm /content/result/screening/tmp.tab.fmt; \
    done; \
  fi

In [ ]:
# screening for similar proteins using FATCAT

%cd /content/input
! if [ $SCREEN = "fatcat" ]; then \
    for f in *; do \
      FATCATSearch.pl $f /content/list_scope.tab -b -i1 /content/input -i2 /content/database | \
      sort -k11nr | \
      head -n $HEADN | \
      perl /content/programs/remolog/scripts/format_result_FATCAT.pl - /content/programs/remolog/data/maxScore_fatcat.tab > /content/result/screening/$f.tab.fmt; \
    done; \
    cat /content/result/screening/*.fmt > /content/result/fatcat_formatted.tab; \
  fi

In [ ]:
# screening for similar proteins using tmalign

%cd /content/input
! if [ $SCREEN = "tmalign" ]; then \
  for f in *; do \
    if [ -f /content/result/screening/${f}.tab ]; then \
      rm /content/result/screening/${f}.tab; \
    fi; \
    for l in $(cat /content/list_scope.tab); \
      do TMalign /content/input/$f /content/database/${l}.pdb | perl /content/programs/remolog/scripts/parser_TMalign.pl - >> /content/result/screening/${f}.tab ; \
      done; \
    sort -k3nr /content/result/screening/${f}.tab | grep ${f} | head -n $HEADN > /content/result/screening/${f}.tab.fmt; \
    done; \
  cat /content/result/screening/*.fmt > /content/result/tmalign_formatted.tab; \
fi

In [ ]:
%cd /content/input
! if [ $SCREEN != "fatcat" ]; then \
  if [ -f /content/result/fatcat_formatted.tab ]; then \
    rm /content/result/fatcat_formatted.tab; \
  fi; \
  for f in *; do \
    for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
      FATCAT -p1 $f -p2 $l.ent.pdb -i1 /content/input -i2 /content/database -b | \
      perl /content/programs/remolog/scripts/format_result_FATCAT.pl - /content/programs/remolog/data/maxScore_fatcat.tab >>  /content/result/fatcat_formatted.tab ; \
    done; \
  done; \
  fi

In [ ]:
%cd /content/input
! if [ $SCREEN != "tmalign" ]; then \
    if [ -f /content/result/tmalign_formatted.tab ]; then \
      rm /content/result/tmalign_formatted.tab; \
    fi; \
    for f in *; do \
      for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
        TMalign /content/input/$f /content/database/$l.ent.pdb | perl /content/programs/remolog/scripts/parser_TMalign.pl - >> /content/result/tmalign_formatted.tab ; \
      done; \
    done;\
  fi

In [ ]:
%cd /content/input
! if [ $SCREEN != "lovoalign" ]; then \
    if [ -f /content/result/lovoalign_formatted.tab ]; then \
      rm /content/result/lovoalign_formatted.tab; \
    fi; \
    for f in *; do \
      for l in $(cut -f2 /content/result/screening/$f.tab.fmt); do \
        lovoalign -p1 /content/input/$f -p2 /content/database/$l.ent.pdb | perl /content/programs/remolog/scripts/parser_lovoalign.pl - >> /content/result/lovoalign_formatted.tab; \
      done; \
    done; \
  fi

In [ ]:
%cd /content/result
! perl /content/programs/remolog/scripts/join_table.pl /content/result/fatcat_formatted.tab /content/result/tmalign_formatted.tab | \
  perl /content/programs/remolog/scripts/join_table.pl - /content/result/lovoalign_formatted.tab > result.tab

## Making prediction and writing the results

In [ ]:
from joblib import load
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

clf = load('/content/programs/remolog/model.joblib')

In [ ]:
header = ["query", "subject",
          "lovo_finalScore", "lovo_coverage", "lovo_rmsd", "lovo_gaps", "lovo_finalScoreNorm",
          "tm_AliLen", "tm_RMSD", "tm_n_ident/n_aln", "tm_TM-score (chain 2)", "tm_d0 (chain 2)",
          "fatcat_subject-len", "fatcat_Twists", "fatcat_ini-len", "fatcat_ini-rmsd", "fatcat_opt-equ", "fatcat_opt-rmsd", "fatcat_chain-rmsd", "fatcat_Score", "fatcat_align-len", "fatcat_Gaps", "fatcat_rel_score", "fatcat_rel_align",
          ]
data = pd.read_csv("/content/result/result.tab", sep="\t", header=None)
data.columns = header

annot = pd.read_csv("/content/programs/remolog/data/dir.cla.scope.2.08-stable_filtered.txt", header=None, sep="\t")
annot[['cl','cf','sf','fa','dm','sp','px']] = annot[5].str.split(',',expand=True)
annot_cl = annot.loc[:,[0,"cl"]].drop_duplicates()
annot_cl2 = np.array(annot_cl["cl"]).reshape(-1,1)
header_cl = list(annot.loc[:,["cl"]].drop_duplicates()["cl"])


In [ ]:
from sklearn import preprocessing

enc = preprocessing.OneHotEncoder(categories='auto')
enc.fit(annot_cl2)
oheCl = pd.DataFrame(enc.transform(annot_cl2).toarray(), columns=header_cl)
data_cl = pd.concat([annot_cl[0], oheCl], axis=1)
data2 = pd.merge(right=data_cl, how="inner", left=data, right_on=0, left_on="subject")
data2

In [ ]:
X = data2.iloc[:,2:]
X.drop(X.columns[[22]], axis=1, inplace=True)
pred = clf.predict(X)
data["pred"] = pred

In [ ]:
annot.index = annot.loc[:, 0]
annot = annot.loc[:,["cl","cf","sf"]]
data = data.join(annot, on="subject")

In [ ]:
data = data.sort_values(by=["pred","fatcat_rel_score"], ascending=False)

In [ ]:
# write and download result file
from google.colab import files

data.to_csv("/content/result/"+jobName+".tab", index=False)
files.download("/content/result/"+jobName+".tab")

In [ ]:
# function to display table
from google.colab import data_table
data_table.enable_dataframe_formatter()

def hyperlink(path):
	
    # returns the substring of a path

    pathList = path.split("=")
    f_url = pathList[len(pathList)-1]
    path="https://scop.berkeley.edu/search/?ver=2.08&key="+f_url
    #print(f_url)
    
    # convert the path into clickable link
    return '<a target="_blank" href="{}">{}</a>'.format(path, f_url)


In [ ]:
# functions to display 3d alignment
! pip install py3Dmol
import py3Dmol
import glob
import matplotlib.pyplot as plt 

In [ ]:
import ipywidgets as widgets
query_list = data.loc[:,"query"].unique()
query_picker = widgets.Dropdown(options=query_list, value=query_list[0])
subject_list = list(range(1,n+1))
subject_picker = widgets.Dropdown(options=subject_list, value=subject_list[0])

#button = widgets.Button(description="Submit")
#output = widgets.Output()
#display(button)

def display_str():
  selected_subject = data[data["query"] == query_picker.value].iloc[subject_picker.value - 1,:]
  os.chdir("/content/view")
  os.system("rm *")
  os.system("FATCAT -p1 "+query_picker.value+" -p2 "+selected_subject[1]+".ent.pdb -i1 /content/input -i2 /content/database -t")

  with open("/content/view/tmp.opt.twist.pdb") as ifile:
      system = "".join([x for x in ifile])
      
  view = py3Dmol.view(width=400, height=300)
  view.addModelsAsFrames(system)

  view.setStyle({'chain':'A'}, {'cartoon':{'color':'blue'}})
  view.setStyle({'chain':'B'}, {'cartoon':{'color':'yellow'}})
  view.zoomTo()
  view.show()


  display(pd.DataFrame(selected_subject))


#button.on_click(on_button_clicked)



## Display table 

In [ ]:
data2 = data.style.format({'subject': hyperlink, 'sf': hyperlink})
data2

In [ ]:
data_table.disable_dataframe_formatter()

Description of the columns
- query: Query name
- subject: Subject name

Lovoalign
- lovo_finalScore: Final score;
- lovo_coverage: Alignment coverage;
- lovo_rmsd: RMSD;
- lovo_gaps: # of gaps;
- lovo_finalScoreNorm: Normalized score;

TM-align
- tm_AliLen: Alignment length;
- tm_RMSD: RMSD;
- tm_n_ident/n_aln: proportion of # identical atom and aligned length;
- tm_TM-score (chain 2): TM-score normalized by subject;
- tm_d0 (chain 2): scale factor used to calculate TM-score; 

FATCAT
- fatcat_subject-len: subject length;
- fatcat_Twists: # of twists;
- fatcat_ini-len: Initial alignment length;
- fatcat_ini-rmsd: Initial RMSD;
- fatcat_opt-equ: # of equivalent positions in the alignment;
- fatcat_opt-rmsd: RMSD of aligned Cα atoms of the input structures with structural rearragement;
- fatcat_chain-rmsd: RMSD of aligned Cα atoms of the input structures without structural rearragement;
- fatcat_Score: Alignment score
- fatcat_align-len: Alignment length 
- fatcat_Gaps: # of gaps in the alignment
- fatcat_rel_score: proportion of the alignment score and maximum score
- fatcat_rel_align: proportion of the # of aligned position with subject length

Prediction
- pred: prediction (0: not remote homolog; 1: remote homolog)

SCOPe annotation
- cl: subject SCOPe class
- cf: subject SCOPe fold
- sf: subject SCOPe superfamily

## Display protein structure alignment

In [ ]:
print("Choose a query")
display(query_picker)
print("Choose the Subject")
display(subject_picker)

In [ ]:
# Choose a query and a subject in the cell above and run this cell to display
# the structure alignement performed by FATCAT.
# chain in blue: Query
# chain in yellow: Subject
display_str()